# N-impulse problem with Sun-B1 dynamics

In [3]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
import joptimise
using Printf

In [12]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")

Main.R3BP

In [13]:
plotly()

Plots.PlotlyBackend()

In [14]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [15]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

tmax_si = 0.3  # N
isp_si = 3500  # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2000  # kg

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

Halo guess Az_km: 1200.0
Linear stability ν = 618.7618470919092


0.0016413295927890758

## Construct optimization problem

In [27]:
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
ωb = 2*pi*(t_synodic - t_sidereal)/t_synodic/t_sidereal*param3b.tstar
ωM = 2*pi / (t_synodic / param3b.tstar)
θM = π  # placeholder
params = [param3b.mu2, param3b.mus, param3b.as, θM, ωM, ωb, 0.0, 0.0, 0.0, 0.0, 0.0]

11-element Vector{Float64}:
      0.01215058560962404
 328900.5598102475
    388.8212386592645
      3.141592653589793
      0.9251999994040079
      0.07480000059599223
      0.0
      0.0
      0.0
      0.0
      0.0

In [45]:
u0_test = [
     387.6985504060269
  -2.7938137447835705e-7
  -5.620290959516347e-23
   1.0294967930424787e-6
  -1.203809978563231
   1.3395041586755617e-31
   1.0
]
_prob_base = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, u0_test, [0,-23.5], params)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, -23.5)
u0: 7-element Vector{Float64}:
 387.6985504060269
  -2.7938137447835705e-7
  -5.620290959516347e-23
   1.0294967930424787e-6
  -1.203809978563231
   1.3395041586755617e-31
   1.0

In [46]:
sol = SailorMoon.integrate_rk4(_prob_base, 0.005);

In [47]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [48]:
moon = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth = plot_circle(param3b.mu2, param3b.as, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0)

2×100 Matrix{Float64}:
 389.981  389.978      389.971    389.96      …  389.978      389.981
   0.0      0.0735331    0.14677    0.219416      -0.0735331   -2.83969e-16

In [49]:
pcart = plot(hcat(sol.u...)[1,:], hcat(sol.u...)[2,:], set_aspect=:equal, size=(700,700))
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

In [50]:
"""
    transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)

Transform state from Earth-Moon rotating frame to Sun-B1 rotating frame.
Careful with sign of ωs!! (should be negative)
"""
function transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)
    ωm = -ωs
    θm = π - θs
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    C = [
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
         ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
         0          0         0 0       0      1
    ]
    state_conv = C * state
    state_conv = state_conv + [param3b.as, 0,0,0,0,0]
    
    return state_conv
end

transform_EMrot_to_SunB1

In [97]:
dt = 0.01
rp_parking = (6378+200)/param3b.lstar

0.01709689063726318

In [147]:
propagate_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false) where T
    # unpack
    nx = length(x)
    θfm, tof, eta, r_apogee, raan, ϕ = x[1:6]  # θfm: Moon angle at final time
    tau1     = x[7 : floor(Int, (nx-6)/2 + 6)]  # discretization numbers are the same for the first & second arc
    tau2     = x[floor(Int, (nx-6)/2 + 7) : end]
    tof_fwd = tof * eta
    tof_bck = tof * (1 - eta)
    
    θfs = π - θfm
    
    # construct initial state
    sma = (rp_parking + r_apogee)/2
    ecc = (r_apogee - rp_parking)/(r_apogee + rp_parking)
    sv0_kep = [sma, ecc, 0.0, raan, 0.0, 0.0]
    println("sv0_kep: $sv0_kep|")
    θ0s = θfs - param3b.oms*(tof_fwd + tof_bck)   # initial Sun angle
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    sv0_em = inertial2rotating(sv0_i, θ0s, 1.0) + [-param3b.mu2,0,0,0,0,0]
    sv0 = vcat(transform_EMrot_to_SunB1(sv0_em, θ0s, param3b.oms), 1.0)
    println("sv0: $sv0")
    θ0m = π - θ0s
    
    # construct final state
    svf_em = SailorMoon.set_terminal_state(ϕ, param3b, LPOArrival)
    svf = vcat(transform_EMrot_to_SunB1(svf_em, θfs, param3b.oms), 1.0)
    
    # initialize storage
    sols_fwd, sols_bck = [], []
    params_fwd, params_bck = [], []
    
    # forward propagation
    nhalf = Int(n/2)
    tspan_fwd = [0, tof_fwd/nhalf]
    for i = 1:nhalf
        τ, γ, β = tau1[3*i-2 : 3*i]
        params = [param3b.mu2, param3b.mus, param3b.as, θ0m, ωM, ωb, τ, γ, β, mdot, tmax]
        _prob = remake(
            _prob_base; tspan=tspan_fwd, 
            u0 = sv0,
            p = params,
        )
        sol = SailorMoon.integrate_rk4(_prob, dt);
        #DifferentialEquations.solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
        if get_sols
            push!(sols_fwd, sol)
            push!(params_fwd, params)
        end
        # update θ0 and sv0
        θ0m += param3b.oms*sol.t[end]
        sv0 = sol.u[end]
    end
    
    # back propagation
    tspan_bck = [0, -tof_bck/(n-nhalf)]
    for i = 1:n-nhalf
        τ, γ, β = tau2[3*i-2 : 3*i]
        params = [param3b.mu2, param3b.mus, param3b.as, θfm, ωM, ωb, τ, γ, β, mdot, tmax]
        _prob = remake(
            _prob_base; tspan=tspan_bck, 
            u0 = svf,
            p = params,
        )
        sol = SailorMoon.integrate_rk4(_prob, dt);
        #sol = DifferentialEquations.solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
        if get_sols
            push!(sols_bck, sol)
            push!(params_bck, params)
        end
        # update θf (note sol.t[end] is negative so += is correct) and svf
        θfm += param3b.oms*sol.t[end]
        svf = sol.u[end]
    end
    
    # residual
    if get_sols == false
        return svf - sv0
    else
        return sols_fwd, sols_bck, params_fwd, params_bck
    end
end

#19 (generic function with 2 methods)

In [148]:
n = 20

20

In [149]:
bounds_tau = [0,1]
bounds_γ   = [-π, π]
bounds_β   = [-π, π]
#  θfm, tof, eta, ra, raan, ϕ = x[1:6]
xtest = [
    π, 23, 0.55, 0.45, 1.5, 0.0
]
for i = 1:n
    xtest = vcat(xtest, [0.1,0,0])
end

lx = [
    2.6, 18, 0.3, 1.9, 0.7, 0.0, -1.0
]
ux = [
    3.2, 27, 0.7, 2.4, 0.995, 2π, 1.0
]
for i = 1:n
    lx = vcat(lx, [0,bounds_γ[1],bounds_β[1]])
    ux = vcat(ux, [0,bounds_γ[1],bounds_β[2]])
end

In [150]:
sols_fwd, sols_bck, params_fwd, params_bck = propagate_trajectory(xtest, true);
#propagate_trajectory(xtest, false)

sv0_kep: [0.2335484453186316, 0.926795099774962, 0.0, 1.5, 0.0, 0.0]|
sv0: [388.8287268010157, -0.011640363155616378, 0.0, 2.294167916853228, 10.290643888594529, 0.0, 1.0]


In [151]:
cs_fwd = palette([:darkred, :navyblue], max(Int(n/2),2))
cs_bck = palette([:darkgreen, :goldenrod], max(n-Int(n/2),2))

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false)

# trajectory
for (ifwd,sol_fwd) in enumerate(sols_fwd)
    plot!(pcart, SailorMoon.Array(sol_fwd)[1,:], SailorMoon.Array(sol_fwd)[2,:], color=cs_fwd[ifwd], 
        linewidth=1.5, label="fwd $ifwd", linestyle=:dashdot)
end
for (ibck,sol_bck) in enumerate(sols_bck)
    plot!(pcart, SailorMoon.Array(sol_bck)[1,:], SailorMoon.Array(sol_bck)[2,:], color=cs_bck[ibck], 
        linewidth=1.5, label="bck $ibck", linestyle=:solid)
end

plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

plot!(pcart; title="Initial guess")
pcart